In [1]:
%load_ext autoreload
%autoreload 2

In [41]:
from utils import get_data, get_conditional_data
from nflows.utils import tensor2numpy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from plot import set_bounds, plot_grid, no_tick_labels

import matplotlib
matplotlib.use('macosx')
import matplotlib.pyplot as plt
plt.switch_backend('macosx')

In [42]:
def get_numpy_data(name, n_points):
    return tensor2numpy(get_data(name, n_points).data)


def get_cond_numpy_data(condition_type, name, n_points, condition=None):
    data_obj = get_conditional_data(condition_type, name, n_points)
    if condition is None:
        data = data_obj.data
        return tensor2numpy(data)        
    else:
        return data_obj._get_conditional(condition)[0]

In [43]:
datasets = [
#         "anulus",
        "ring",
        "concentricrings",
        "fourcircles",
#         "checkerboard",
#         "spirals",
#         "star",
#         "eightstar",
    ]


n_points = int(1e4)
paths = {}
for dd in datasets:
    data = get_numpy_data(dd, n_points)
    path = f'results/dummy/{dd}.h5'
    paths[dd] = path
    pd.DataFrame(data, columns=['left', 'right']).to_hdf(path, 'f4f')

In [44]:
paths

{'ring': 'results/dummy/ring.h5',
 'concentricrings': 'results/dummy/concentricrings.h5',
 'fourcircles': 'results/dummy/fourcircles.h5'}

In [45]:
# Make a fake from_to_target dictionary that can be used to build a grid of images
grid = {}
for inp in datasets:
    for target in datasets:
        grid[f'{inp}_to_{target}'] = paths[target]

In [46]:
import pathlib
n_points = int(1e4)
nm = pathlib.Path('results/grid.pdf')
key = 'f4f'
columns = ['left', 'right']
grid

{'ring_to_ring': 'results/dummy/ring.h5',
 'ring_to_concentricrings': 'results/dummy/concentricrings.h5',
 'ring_to_fourcircles': 'results/dummy/fourcircles.h5',
 'concentricrings_to_ring': 'results/dummy/ring.h5',
 'concentricrings_to_concentricrings': 'results/dummy/concentricrings.h5',
 'concentricrings_to_fourcircles': 'results/dummy/fourcircles.h5',
 'fourcircles_to_ring': 'results/dummy/ring.h5',
 'fourcircles_to_concentricrings': 'results/dummy/concentricrings.h5',
 'fourcircles_to_fourcircles': 'results/dummy/fourcircles.h5'}

In [47]:
plot_grid(grid, columns, nm)

#  Making conditional shifting plots

In [14]:
data_path = pathlib.Path('results/copied/rotating/four_circles/eval_data.h5')
with pd.HDFStore(data_path) as hdf:
    keys = hdf.keys()
    
data = {}
for key in keys:
    data[key] = pd.read_hdf(data_path, key=key)

In [15]:
keys

['/base_dist',
 '/f4f_0_17',
 '/f4f_0_33',
 '/f4f_0_50',
 '/f4f_0_67',
 '/f4f_0_83',
 '/f4f_1_00']

In [53]:
data[keys[0]].head()

,cond_0_17_x,cond_0_17_y,cond_0_33_x,cond_0_33_y,cond_0_50_x,cond_0_50_y,cond_0_67_x,cond_0_67_y,cond_0_83_x,cond_0_83_y,cond_1_00_x,cond_1_00_y
0,1.953567,-0.067639,0.402605,-0.126658,1.582242,-0.870633,-2.440760,-0.206228,-0.779793,-1.547221,1.180737,1.762966
1,0.319603,-0.459517,1.288053,-0.893241,-1.660053,0.897860,-2.389606,-0.562821,1.501377,-1.141023,2.021014,-0.776158
2,1.317849,-2.023931,-1.859925,0.473827,-0.797248,0.991506,0.798813,-1.047505,1.277512,-1.210241,-0.986413,-1.901521
3,-0.781796,-1.313114,0.703081,-0.039403,-1.606169,0.427412,-0.196846,1.011672,0.745172,-0.081470,-2.146607,-0.826128
4,-1.971639,0.860941,-1.645433,0.610468,-0.402735,-1.740190,2.039521,-0.583985,1.525279,-1.332822,-1.058404,-1.886294


In [17]:
data[keys[1]].head()

,input_x,input_y,left_cond,right_cond,transformed_x,transformed_y,left_enc_x,left_enc_y,base_transfer_x,base_transfer_y
0,-0.682852,0.979322,0.0,0.166667,-0.549286,1.056929,0.738976,0.014081,-0.557524,1.053260
1,0.895165,-0.671559,0.0,0.166667,0.802765,-0.783456,-0.456827,0.536372,0.897547,-0.790018
2,0.607479,-0.691070,0.0,0.166667,0.514020,-0.762939,-0.412917,0.416268,0.536918,-0.742103
3,1.999558,0.426514,0.0,0.166667,2.040847,0.161854,1.190064,2.003976,2.064592,0.272072
4,0.625424,1.917708,0.0,0.166667,0.871498,1.821582,0.909688,1.848931,1.169721,1.743510


In [20]:
grid = {'fourcircles': data_path}
keys = ['/f4f_0_17', '/f4f_0_33', '/f4f_0_50', '/f4f_0_67', '/f4f_0_83', '/f4f_1_00']
columns = ['transformed_x', 'transformed_y']
n_points = int(1e6)
nm = pathlib.Path('results/conditional_grid.pdf')


grid = {'fourcircles': data_path}
keys = ['/f4f_0_17', '/f4f_0_33', '/f4f_0_50', '/f4f_0_67', '/f4f_0_83', '/f4f_1_00']
columns = ['base_transfer_x', 'base_transfer_y']
n_points = int(1e6)
nm = pathlib.Path('results/conditional_bd_transform.pdf')

In [21]:
def add_2d_hist(axis, data):
    axis.hist2d(data[:, 0], data[:, 1], bins=128)
    set_bounds(axis)
    
    
def add_2d_hist(ax, data, bins=50, bounds=None, clip_val=4):
    count, xbins, ybins = np.histogram2d(data[:, 0], data[:, 1], bins=bins)
    count[count == 0] = np.nan
    if clip_val is None:
        clip_val = np.nanquantile(count, 0.01)
    ax.imshow(count.T,
              origin='lower', aspect='auto',
              extent=[xbins.min(), xbins.max(), ybins.min(), ybins.max()],
              vmin=clip_val
              )
    set_bounds(ax, clip_val)

N_inputs = len(grid)
N_targets = len(keys)

# Add one because we want to plot the data around the perimeter.
fig, ax = plt.subplots(N_inputs, N_targets + 1,
                       figsize=(5 * (N_targets + 1), 5 * N_inputs))

if ax.ndim == 1:
    ax = ax[np.newaxis, ...]

for i, inp in enumerate(grid):
    data = pd.read_hdf(data_path, key=keys[0])[['input_x', 'input_y']].to_numpy()
    add_2d_hist(ax[i, 0], data)

    for j, key in enumerate(keys):
        data = pd.read_hdf(data_path, key=key)[columns].to_numpy()
        add_2d_hist(ax[i, j + 1], data)

# add a big axis, hide frame
fig.add_subplot(111, frameon=False)
# hide tick and tick label of the big axis
plt.tick_params(labelcolor='none', which='both', top=False, bottom=False, left=False, right=False)
plt.ylabel("Input", fontsize=40)
plt.title("Target", fontsize=40, pad=30)

fig.tight_layout()
fig.savefig(nm)

# making plot comparing base dist to flows for flow

In [26]:
# Flow for flow model
plt.rcParams['text.usetex'] = True
grid = {'fourcircles': data_path}
keys = ['/f4f_0_17', '/f4f_0_33', '/f4f_0_50', '/f4f_0_67', '/f4f_0_83', '/f4f_1_00']
keys = ['/f4f_0_33', '/f4f_0_67', '/f4f_1_00']
columns = ['transformed_x', 'transformed_y']
n_points = int(1e6)
nm = pathlib.Path('results/f4f_compare.pdf')
clip_val = 3
data_path = pathlib.Path('results/copied/rotating/four_circles/eval_data.h5')

N_inputs = len(grid) + 2
N_targets = len(keys)

# Add one because we want to plot the data around the perimeter.
fig, global_axes = plt.subplots(N_inputs, N_targets,
                       figsize=(5 * (N_targets), 5 * N_inputs))

# Show the data that is being targeted
ax = global_axes[0]

if ax.ndim == 1:
    ax = ax[np.newaxis, ...]

for i, inp in enumerate(grid):
    for j, key in enumerate(keys):
        rotation = float(key.split('_', 1)[-1].replace('_', '.')) * 45
        data = get_cond_numpy_data('rotation', 'fourcircles', n_points, 
                                   condition=rotation)
        add_2d_hist(ax[i, j], data, clip_val=clip_val)
        ax[i, j].set_title(fr'${rotation:.0f}^\circ$', fontsize=40)
        no_ticks(ax[i, j])
        
ax[0, 0].set_ylabel('Data', fontsize=40)

# Look at the base density samples
ax = global_axes[1]

if ax.ndim == 1:
    ax = ax[np.newaxis, ...]

for i, inp in enumerate(grid):
    for j, key in enumerate(keys):
        data = pd.read_hdf(data_path, key=key)[columns].to_numpy()
        add_2d_hist(ax[i, j], data, clip_val=clip_val)
#         rotation = float(key.split('_', 1)[-1].replace('_', '.')) * 45
#         ax[i, j].set_title(fr'${rotation:.0f}^\circ$', fontsize=40)
        no_ticks(ax[i, j])
        
ax[0, 0].set_ylabel('Flow4Flow', fontsize=40)


# Base desnity only
# Settings
key = '/base_dist'
# columns = [[f'cond_{cond}_x', f'cond_{cond}_y'] for cond in 
#            ['0_17', '0_33', '0_50', '0_67', '0_83', '1_00']]
columns = [[f'cond_{cond}_x', f'cond_{cond}_y'] for cond in 
           [key.split('_', 1)[-1] for key in keys]]
# This was produced locally by loading the base density but not the flow4flow model
data_path = pathlib.Path('results/copied/rotating/four_circles/base_density_eval_data.h5')

# Script
N_inputs = len(grid)
N_targets = len(keys)

ax = global_axes[2]

if ax.ndim == 1:
    ax = ax[np.newaxis, ...]

for i, inp in enumerate(grid):
    for j, clmns in enumerate(columns):
        data = pd.read_hdf(data_path, key=key)[clmns].to_numpy()
        add_2d_hist(ax[i, j], data, clip_val=clip_val, bins=100)
        no_ticks(ax[i, j])
        
ax[0, 0].set_ylabel('Base Density', fontsize=40)

fig.tight_layout()
fig.savefig(nm)
plt.rcParams['text.usetex'] = False

In [27]:
x

NameError: name 'x' is not defined

# Color grids

In [10]:
from plot import make_colormap, assign_colors, add_scatter
import torch
from scipy.stats import binned_statistic_2d

In [29]:
# Flow for flow model
plt.rcParams['text.usetex'] = True
grid = {'fourcircles': data_path}
keys = ['/f4f_0_17', '/f4f_0_33', '/f4f_0_50', '/f4f_0_67', '/f4f_0_83', '/f4f_1_00']
keys = ['/f4f_0_33', '/f4f_0_67', '/f4f_1_00']
n_points = int(1e6)
clip_val = 3

nm = pathlib.Path('results/f4f_color_compare_four_circles.pdf')
data_path = pathlib.Path('results/copied/rotating/four_circles/eval_data.h5')

clip_val = 2.5
nm = pathlib.Path('results/f4f_color_compare_star.pdf')
data_path = pathlib.Path('results/copied/rotating/star/eval_data.h5')

N_inputs = len(grid) + 1
N_targets = len(keys) + 1

# Add one because we want to plot the data around the perimeter.
fig, ax = plt.subplots(N_inputs, N_targets,
                       figsize=(5 * (N_targets), 5 * N_inputs))

# Look at the base density samples

if ax.ndim == 1:
    ax = ax[np.newaxis, ...]
    
n_points = int(1e6)
indx = torch.randperm(int(1e6))
    
def add_color_axis(ax, df, columns, colors=None, bins=200):
    data = torch.Tensor(df[columns].to_numpy())[indx][:n_points]
    if colors is None:
        img = make_colormap()
        colors = assign_colors(img, data)
    add_scatter(ax, data, colors, clip_val=clip_val)
    no_ticks(ax)
    return colors
    
# Plot the input distributions
df = pd.read_hdf(data_path, key=keys[0])
colors = add_color_axis(ax[0, 0], df, ['input_x', 'input_y'])
add_color_axis(ax[1, 0], df, ['input_x', 'input_y'], colors=colors)
ax[0, 0].set_title('Input Data', fontsize=40)
ax[0, 0].set_ylabel('Flow4Flow', fontsize=40)
ax[1, 0].set_ylabel('Base Distribution', fontsize=40)

for j, key in enumerate(keys):
    df = pd.read_hdf(data_path, key=key)
    # flow for flow
    add_color_axis(ax[0, j + 1], df, ['transformed_x', 'transformed_y'], colors=colors)
    # BD transfer
    add_color_axis(ax[1, j + 1], df, ['base_transfer_x', 'base_transfer_y'], colors=colors)
    rotation = float(key.split('_', 1)[-1].replace('_', '.')) * 45
    ax[0, j + 1].set_title(fr'${rotation:.0f}^\circ$', fontsize=40)

fig.tight_layout()
fig.savefig(nm)
plt.close(fig)
plt.rcParams['text.usetex'] = False
print('Saved') 


Saving
Saved


In [32]:
from plot import conditional_color_grid


keys = ['/f4f_0_17', '/f4f_0_33', '/f4f_0_50', '/f4f_0_67', '/f4f_0_83', '/f4f_1_00']
keys = ['/f4f_0_33', '/f4f_0_67', '/f4f_1_00']

clip_val = 2.5
nm = pathlib.Path('results/f4f_color_compare_star.pdf')
data_path = pathlib.Path('results/copied/rotating/star/eval_data.h5')
conditional_color_grid(data_path, keys, nm, clip_val)

In [128]:
np.sum(~np.isnan(pixelated[np.isnan(pixelated).any(-1)]))

0

In [110]:
pixelated[..., np.isnan(pixelated).any(-1)]

IndexError: boolean index did not match indexed array along dimension 2; dimension is 3 but corresponding boolean dimension is 50

In [78]:
data = df[['input_x', 'input_y']].to_numpy()[indx.numpy()][:n_points]

In [80]:
fig, ax = plt.subplots()

plt.imshow(pixelated / 255)
print('Saving')
plt.savefig('results/f4f_color_compare_star.png')
plt.close(fig)

Saving


In [72]:
pixelated[np.isnan(pixelated)] = 0

# Checking spline initialisation

In [87]:
from utils import get_activation, spline_inn
import torch
from plot import plot_arrays

In [88]:
sv_dir = pathlib.Path('results/initialisations/')
sv_dir.mkdir(exist_ok=True)

In [89]:
# Using actual data it is easy to tell, but not so much with a gaussian
data = torch.Tensor(data)

In [94]:
data = torch.randn(2 * int(1e5)).view(-1, 2)

In [95]:
mse_dist = []
for i in range(10):
    
    transformer = spline_inn(2,
                          nodes=128,
                          num_blocks=2,
                          num_stack=4,
                          tail_bound=4.0,
                          activation=get_activation('leaky_relu'),
                          num_bins=8,
                          context_features=1,
                          flow_for_flow=True
                          )
    
    with torch.no_grad():
        trasnformed, _ = transformer(data)
    mse_dist += [(((trasnformed - data) ** 2) ** 0.5).sum(-1).mean()]
    plot_arrays({
        'Input': data,
        'Transform': trasnformed
    }, sv_dir, f'run_{i}')

In [96]:
mse_dist

[tensor(1.1199),
 tensor(2.1793),
 tensor(1.6284),
 tensor(1.7033),
 tensor(1.9216),
 tensor(1.1970),
 tensor(0.8968),
 tensor(0.8992),
 tensor(1.6787),
 tensor(1.2256)]